In [ ]:
## add in security measures

In [ ]:
import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

## See Crossing Inventory Data Search for more information. [Link]('https://railroads.dot.gov/safety-data/fra-safety-data-reporting/crossing-inventory-data-search?title=&field_topic_target_id=1311')

Metadata: https://railroads.dot.gov/sites/fra.dot.gov/files/fra_net/3088/datbyfld22802.pdfhttps://railroads.dot.gov/sites/fra.dot.gov/files/fra_net/3088/datbyfld22802.pdf

In [4]:
import urllib
import pandas
import json
import numpy
import os
import sys
import geopandas
import janitor
from path import Path
import folium
import matplotlib.pyplot as plt
import urllib3
import requests

In [6]:
# path configuration
working_directory = Path.getcwd()

inputs_path = working_directory / 'inputs'
outputs_path = working_directory / 'outputs'

In [7]:
# export function
def to_file(callback, desired_name, extension='csv'):
    current_files = sorted(outputs_path.files(desired_name + '*.' + extension))
    if current_files:
        last_file = current_files[-1]
        os.remove(last_file)
    final_name = '{}.{}'.format(desired_name, extension)
    callback(outputs_path / final_name)

In [8]:
# Define the number of items to retrieve per API call
items_per_call = 10

# Initialize a variable to keep track of the current skip value
current_skip = 0

total_items_to_retrieve = 200

# Create an empty list to store the data
data_list = []

### Set up API parameters

In [26]:
"https://safetydata.fra.dot.gov/MasterWebService/PublicApi/frads/v1/odata/gcis/Crossings?token=7d9f06fbf611c46d2d767ebd72742923&$format=json&$filter=StateCD eq '29'&$skip=110"

"https://safetydata.fra.dot.gov/MasterWebService/PublicApi/frads/v1/odata/gcis/Crossings?token=7d9f06fbf611c46d2d767ebd72742923&$format=json&$filter=StateCD eq '29'&$skip=110"

In [27]:
endpoint

"https://safetydata.fra.dot.gov/MasterWebService/PublicApi/frads/v1/odata/gcis/Crossings?&token=7d9f06fbf611c46d2d767ebd72742923&$format=json&$filter=StateCD eq '29'&$skip=110"

In [29]:
# set up api parameters
base_url = "http://safetydata.fra.dot.gov/MasterWebService/PublicApi/frads/v1/odata/gcis"
dataset = 'Crossings'
state_fips = "StateCD eq '29'"
api_key = '7d9f06fbf611c46d2d767ebd72742923'

# by agency ID
agency = 'ReportingAgencyId eq 891'
# by county
county_fips = "CntyCD eq '29037'"
# by railroad
railroad = "OperatingRailroadCode eq 'AM'"

### By State

In [31]:
# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?token={api_key}&$format=json&$filter={state_fips}&$skip={current_skip}"

    # Make the API call here
    response = get_legacy_session().get(endpoint, verify=False)

    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pandas.DataFrame(data_list)

C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'safetydata.fra.dot.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'safetydata.fra.dot.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'safetydata.fra.dot.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warn

### By Reporting Agency

In [ ]:


# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?&token={api_key}&$format=json&$filter={agency}&$skip={current_skip}"
    # Make the API call here
    response = requests.get(endpoint, verify=False)
    
    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pandas.DataFrame(data_list)

### By county

In [ ]:
# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?&token={api_key}&$format=json&$filter={county_fips}&$skip={current_skip}"

    # Make the API call here
    response = requests.get(endpoint)
    
    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pandas.DataFrame(data_list)

### By Railroad

In [ ]:
# Create a loop to make API calls until you reach the total_items_to_retrieve
while current_skip < total_items_to_retrieve:
    # Build the API endpoint with the current skip value
    endpoint = f"{base_url}/{dataset}?&token={api_key}&$format=json&$filter={railroad}&$skip={current_skip}"

    # Make the API call here
    response = requests.get(endpoint)
    
    # Check if the API call was successful
    if response.status_code == 200:
        # Convert the API response data to a DataFrame
        api_data = response.json().get('value', [])  # Extract 'value' key
        
        # Extend the data_list with the extracted data
        data_list.extend(api_data)
        
        # Increment the skip value for the next iteration
        current_skip += items_per_call
    else:
        print(f"API request failed with status code {response.status_code}")
        break  # Exit the loop if the API request fails

# Create a DataFrame from the data_list
df = pandas.DataFrame(data_list)

In [ ]:
# drop any instance null values from df return for Latitude OR Lnogitude
df = df[~df.Latitude.isna() | ~df.Longitude.isna()]

In [ ]:
 my_geoseries.set_crs(epsg=4326)

In [ ]:
#make points spatial
crs="EPSG:4326"

points = geopandas.GeoDataFrame(
    df, crs=crs, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude)
)

# make points float data type
points[['Latitude','Longitude']] = points[['Latitude','Longitude']].astype(float)

In [ ]:
# narrow down to zone_points-  290376360 is pleasant hill, 290377600 is stratsburg
zone_points = points[points.CityCD.str.contains('290376360|290377600')]

In [ ]:
%store zone_points

In [ ]:
# Latitude and Longitude coordinates for Missouri
missouri_coordinates = [38.573936, -92.603760]

# Create a Folium map centered around Missouri
m = folium.Map(location=missouri_coordinates, zoom_start=7)

In [ ]:
folium.Map(location=geopandas.GeoDataFrame(amtrack_routes.geometry + zone_points.geometry), zoom_start=7)

In [ ]:
# read in amtrack routes
amtrack_routes = geopandas.read_file(inputs_path/'Amtrak_Routes.geojson')

amtrack_routes.head()

In [ ]:
amtrack_routes.geometry.min()

In [ ]:
amtrack_routes.NAME.unique()

# filter for particular route and set crs
filtered_routes = amtrack_routes[amtrack_routes.NAME == 'Kansas City - St. Louis (Missouri River Runner)']
filtered_routes = filtered_routes.set_crs(4326, allow_override=True)

In [ ]:
# filtered points
filtered_routes

In [ ]:
filtered_routes.plot()

In [ ]:
m = folium.Map(location=[your_initial_latitude, your_initial_longitude], zoom_start=your_initial_zoom)


In [ ]:
# Iterate through the GeoDataFrame and add markers for points
for idx, row in zone_points.iterrows():
    # Get the coordinates of the point
    lon, lat = row.geometry.coords[0]
    # Create a marker for each point and add it to the map
    folium.Marker([lat, lon],icon=folium.Icon(icon="cloud")).add_to(m)

# Iterate through the GeoDataFrame and add PolyLines for each LineString geometry
for idx, row in filtered_routes.iterrows():
    # Extract the geometry
    geometry = row.geometry

    # Check if it's a MultiLineString
    if geometry.geom_type == 'MultiLineString':
        for line_string in geometry:
            # Convert each LineString to a list of coordinate pairs
            coordinates = list(line_string.coords)
            
            # Create a PolyLine using the coordinates and add it to the map
            folium.PolyLine(locations=coordinates, color='blue').add_to(m)
    elif geometry.geom_type == 'LineString':
        # Convert the LineString to a list of coordinate pairs
        coordinates = list(geometry.coords)

        # Create a PolyLine using the coordinates and add it to the map
        folium.PolyLine(locations=coordinates, color='blue').add_to(m)
        
# Save the map to an HTML file or display it in a Jupyter Notebook
m.save("map.html")
m

In [ ]:
# Assuming you have a GeoDataFrame called "points" with a "geometry" column containing Point geometries
for idx, row in zone_points.iterrows():
    # Get the coordinates of the point
    lon, lat = row.geometry.coords[0]
    # Create a marker for each point and add it to the map
    folium.Marker([lat, lon],icon=folium.Icon(icon="cloud")).add_to(m)
    
# Save the map to an HTML file or display it in a Jupyter Notebook
m

In [ ]:
callback = lambda p: filtered_routes.to_file(p, driver='ESRI Shapefile')
to_file(callback, 'MoDOT_XX_Amtrak_Route', 'shp')

In [ ]:
callback = lambda p: zone_points.to_file(p, driver='ESRI Shapefile')
to_file(callback, 'MoDOT_XX_Amtrak_Points', 'shp')